# Make Recommendation

In [1]:

from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy.distance
import geocoder
import os
from dotenv import load_dotenv, find_dotenv
import time
import seaborn as sns
import numpy as np
import folium
from folium.features import DivIcon
from hdbscan import HDBSCAN
walknum=0
DEBUG=0

In [2]:
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.preprocessing import (
    StandardScaler, LabelBinarizer, Imputer, FunctionTransformer,PolynomialFeatures, OrdinalEncoder
)
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from copy import copy

In [3]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import googlemaps
import os
#from sqlalchemy import create_engine
from dotenv import load_dotenv, find_dotenv
import math
import pickle

In [4]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='../logs/walk_stats.csv',level=logging.DEBUG)
logging.debug("walknum, sim_method, optimzer, max_dist_hr, starting_lat, starting_long, duration, num_points, total_dist, avg_dist,max_dist, stop_order, num_art_stops, num_building_stops, num_plaque_stops, stop_ids, mean_similarity, max_similarity, user_profile, user_prefs")

In [5]:
def make_session():
    db=connect_db() #establish connection / creates database on first run
    Session = sessionmaker(bind=db)
    session = Session()
    return db, session

In [6]:
def add_features(df):
    df['build_century'] = df['build_year_clean'].apply(lambda x: x//100*100)
#     df['cleaned_year']=df['build_year'].apply(lambda x: clean_build_year(x))
#     df['cleaned_year']=pd.to_numeric(df['cleaned_year'],errors='coerce',downcast='integer')
#     df['build_decade']= df['cleaned_year'].apply(lambda x: x//10*10 )
#     df['poi_type_simple'] = df['poi_type'].apply(lambda x: make_simple_poi(x))
    return df

In [7]:
def get_pois_as_df():
    
    db, session = make_session()
    sql='''SELECT poi.*, styl.style, pcat.category
    FROM points_of_interest poi
    LEFT JOIN architectural_styles styl on (styl.poi_id = poi.poi_id)
    LEFT JOIN poi_categories pcat on (pcat.poi_id = poi.poi_id)
    order by poi.poi_id
    '''
    df = pd.read_sql_query(sql, db)
    df= add_features(df)
    session.close()
    return df

In [8]:
poi_mapper = DataFrameMapper([
   #('Date',None),
    # drop block and address
   # (['poi_id'], None),
   # ('build_decade',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
    ('build_century',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
   #  ('category',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
     #('architect_name',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
     #('style',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
     ('poi_type_simple',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
    # ('current_use',[CategoricalImputer(replacement="n/a"), LabelBinarizer()]),
  #  (['latitude'],None),
  #  (['longitude'],None)
], df_out=True)

In [46]:
def get_user_profile(df_cols, user_sel_prefs):
    '''
    df_cols: list with cols of "vectorized" dataframe
    user_sel_prefs: list with selected cols that interest user
    '''
    
    user_prefs = np.zeros(len(df_cols))
    
    for pref in user_sel_prefs:
        indx=df_cols.index(pref)
        user_prefs[indx] = 1

    df_user=pd.DataFrame(user_prefs).T
    return df_user

In [47]:
# METHOD 2
# ['build_century_1700.0', 'build_century_1800.0', 'build_century_1900.0',
#        'build_century_2000.0', 'poi_type_simple_Art',
#        'poi_type_simple_Building', 'poi_type_simple_Plaque']

def get_user_profile_old(profile_num):
    '''
    # USER PROFILE 1 - interested in People (plaques), Entrepreneurs (32) Medicine (44), Science and Technology (52), Women (56), Sports(53) and modern buildings
    # USER PROFILE 2 -- interested in People (plaques 104) Explorers (33), Pioneers (46), Cemeteries (25), First Nations(35)
# and early buildings (0-9)
    # USER PROFILE 3 -- interested in Art ()
    '''
    user_prefs = np.zeros(7)
    if profile_num== 1:
        for i in [1,5]:
            user_prefs[i] = 1
    elif profile_num==2:
        for i in [2,6]:
            user_prefs[i] = 1
    else:
        for i in [4]:
            user_prefs[i] = 1
    df_user=pd.DataFrame(user_prefs).T
    return df_user

In [11]:
# METHOD 1
# def get_user_profile(profile_num):
#     '''
#     # USER PROFILE 1 - interested in People (plaques), Entrepreneurs (32) Medicine (44), Science and Technology (52), Women (56), Sports(53) and modern buildings
#     # USER PROFILE 2 -- interested in People (plaques 104) Explorers (33), Pioneers (46), Cemeteries (25), First Nations(35)
# # and early buildings (0-9)
#     # USER PROFILE 3 -- interested in Art ()
#     '''
#     user_prefs = np.zeros(105)
#     if profile_num== 1:
#         for i in [32, 52, 44, 56,53, 67,70,75,77, 79, 80, 83, 89, 104]:
#             user_prefs[i] = 1
#     elif profile_num==2:
#         for i in [0,1,2,3,4,5,6,7,8,9,25, 33,35, 46,104]:
#             user_prefs[i] = 1
#     else:
#         for i in [102]:
#             user_prefs[i] = 1
#     df_user=pd.DataFrame(user_prefs).T#, columns=df_features.columns)
# #     df_user.columns = df_features.columns
# #     #df_user=pd.DataFrame.from_records(user_prefs,)
# #     df_user.head()
#     return df_user


In [12]:

def find_similarity(df_features, df_user,  df_poi, sim_method=cosine_similarity):
    '''
    INPUTS: "vectorized" dataframes of POIS and User
    sim_method:
        * cosine_similarity=
        * spatial = 
        * hamming
    RETURNS: dataframe of Points of Interest sorted from highest to lowest similarity rating to the user prefs
    '''
    df_poi['sim_rating'] = 0
    if sim_method =='cosine_similarity':
        print('cosine_similarity')
        cosine_sim = cosine_similarity(df_features,df_user)
        user_matches = pd.DataFrame(cosine_sim, columns=['user_match']) # convert to df for ease
        user_matches.sort_values('user_match', ascending=False, inplace=True) # sort from best to worst matches
       # for ix,row in user_matches.iloc[0:20,:].iterrows():
        for ix,row in user_matches.iterrows():
            # now find matches close to target of interest - ix is row in dataframe that matches to user match
            df_poi.loc[ix,'sim_rating'] = row.user_match
    else:
        
        # spatial method requires numpy arrays
        np_features = df_features.as_matrix()
        U=df_user.as_matrix()
        weight_of_importance=[0.05,0.05,0.05,0.05,.27,.26,.27]
        site_prefs=[]
        for i in range(0,len(np_features)):
            np_features[i,:]
            if sim_method== 'spatial':
               # print('spatial')
                result = spatial.distance.cosine(np_features[i,:], U, weight_of_importance)
            else:
                #print('hamming')
                result = spatial.distance.hamming(np_features[i,:], U)
            res_dict = {'ix': i, 'dist': result}
            site_prefs.append(res_dict)
        # convert similarity rating for each POI to a df
        df_site_prefs = pd.DataFrame(site_prefs)
        df_site_prefs.sort_values('dist', ascending=False, inplace=True)
        
        for ix,row in df_site_prefs.iterrows():
            # now find matches close to target of interest
            df_poi.loc[ix,'sim_rating'] = row.dist


    df_poi.sort_values('sim_rating', inplace=True, ascending=False)
    return df_poi


In [13]:
def find_points_in_area(df, lat, long, num_points, max_distance, inflate_amount = 1.4):
    '''
    Option 2: find a "buffer" of points beyond the specified desired number of stops and then use clustering on the returned matches
     to reduce the list back down to num_points
    Loop through POI dataframe that has already been sorted by similiarty to user prefs (from highest to lowest match)
    find distance for each POI from starting point in meters
    Check if is within the desired radius of the starting point to be a candidate stop
    --> if yes, collect it in the avail_points list and increment the count of found stops
    We have a few POIs with the exact same coordinates (sometimes due to duplicates, but also because some buildings may have multiple artworks)
    --> this causes problems for google maps plotting of stops and also a fairly pointless walk with a bunch of stops in the same area
    --> for now, only include the first POI at a given set of coordinates.  Use sets to make sure our coordinates are unique
    due to some dirty data, have a few POIs with the same coordinates so skip repeats )
    Once we have found the desired number of stops, exit
    
    RETURNS: filtered dataframe of candidate stops with ~length of num_points
    
    ## TODO: colllect all possible stops and then cluster?
    
    '''
    
    avail_points = []
    found_points =0
    prev_coord = (0,0) # sometimes have two points at same lat/long -- don't count as an extra stop
    prev_coords = set({})
    buffered_num_pts = num_points * inflate_amount
    
    
    
    for ix, row in df.iterrows():

        curr_pt = geopy.distance.geodesic((row['latitude'], row['longitude']), (lat, long)).meters
        
        if curr_pt<= max_distance:
            c=(row['latitude'], row['longitude'])
            coord = set({c})
            if prev_coords.issuperset(coord) == False:
                # only include POIs whose coordinates are not already in our list of stops
                prev_coords.add(c)    
                my_dict={}
                my_dict =row.to_dict()
                my_dict['dist_start'] = curr_pt
                avail_points.append(my_dict)
                found_points +=1
                if found_points > buffered_num_pts:
                    break
    df_2 = pd.DataFrame(avail_points)
    return df_2

In [14]:
# OPTION 1 - find exact number of points, even if they may be scattered all over the radius

# def find_points_in_area(df, lat, long, num_points, max_distance):
#     '''
#     Loop through POI dataframe that has already been sorted by similiarty to user prefs (from highest to lowest match)
#     find distance for each POI from starting point in meters
#     Check if is within the desired radius of the starting point to be a candidate stop
#     --> if yes, collect it in the avail_points list and increment the count of found stops
#     We have a few POIs with the exact same coordinates (sometimes due to duplicates, but also because some buildings may have multiple artworks)
#     --> this causes problems for google maps plotting of stops and also a fairly pointless walk with a bunch of stops in the same area
#     --> for now, only include the first POI at a given set of coordinates.  Use sets to make sure our coordinates are unique
#     due to some dirty data, have a few POIs with the same coordinates so skip repeats )
#     Once we have found the desired number of stops, exit
    
#     RETURNS: filtered dataframe of candidate stops with ~length of num_points
    
#     ## TODO: colllect all possible stops and then cluster?
    
#     '''
    
#     avail_points = []
#     found_points =0
#     prev_coord = (0,0) # sometimes have two points at same lat/long -- don't count as an extra stop
#     prev_coords = set({})
#     for ix, row in df.iterrows():

#         curr_pt = geopy.distance.geodesic((row['latitude'], row['longitude']), (lat, long)).meters
        
#         if curr_pt<= max_distance:
#             c=(row['latitude'], row['longitude'])
#             coord = set({c})
#             if prev_coords.issuperset(coord) == False:
#                 # only include POIs whose coordinates are not already in our list of stops
#                 prev_coords.add(c)    
#                 my_dict={}
#                 my_dict =row.to_dict()
#                 my_dict['dist_start'] = curr_pt
#                 avail_points.append(my_dict)
#                 found_points +=1
#                 if found_points > num_points:
#                     break
#     df_2 = pd.DataFrame(avail_points)
#     return df_2

In [15]:
# def find_points_in_area(df, lat, long, num_points, max_distance):
#     '''
#     Loop through POI dataframe that has already been sorted by similiarty to user prefs (from highest to lowest match)
#     find distance for each POI from starting point in meters
#     Check if is within the desired radius of the starting point to be a candidate stop
#     --> if yes, collect it in the avail_points list and increment the count of found stops
#     (due to some dirty data, have a few POIs with the same coordinates so make sure only count that coordinates once)
#     Once we have found the desired number of stops, exit
    
#     RETURNS: filtered dataframe of candidate stops with ~length of num_points
    
#     ## TODO: colllect all possible stops and then cluster?
    
#     '''
    
#     avail_points = []
#     found_points =0
#     prev_coord = (0,0) # sometimes have two points at same lat/long -- don't count as an extra stop
    
#     for ix, row in df.iterrows():

#         curr_pt = geopy.distance.geodesic((row['latitude'], row['longitude']), (lat, long)).meters
        
#         if curr_pt<= max_distance:
#             my_dict={}
#             my_dict =row.to_dict()
#             my_dict['dist_start'] = curr_pt
#             avail_points.append(my_dict)
#             if (row['latitude'], row['longitude']) !=prev_coord:
#                 found_points +=1
#             prev_coord = (row['latitude'], row['longitude'])
#             if found_points > num_points:
#                 break
#     df_2 = pd.DataFrame(avail_points)
#     return df_2

In [41]:
def cluster_stops(df, num_points):
    # get our unique lats and longs of the traps and cluster them
    points = df[['latitude', 'longitude']].drop_duplicates().values

    # convert to radians for HDBScan (per docs)
    rads = np.radians(points)

    clusterer = HDBSCAN(min_cluster_size=int(num_points), metric='haversine',allow_single_cluster=True)
    cluster_labels = clusterer.fit_predict(points)
    
    # how many clusters do we have?
    print(cluster_labels.max())
    
    # add label column to df
    df['labels']=cluster_labels
    print(len(df[df['labels']==0]), len(df[df['labels']!=0]))
    
#     # plot so we can visualize what stops are being dropped
#     my_plot = plot_stops_cluster(starting_lat, starting_long, df)
    
    # drop points not included in cluster
    df = df[df['labels']==0]
    
    # drop labels column
    df.drop(columns='labels', inplace=True)
    return df#, my_plot

In [17]:
#initialize map with a default lat and long
def plot_stops(starting_lat, starting_long, df):
    map_clusters = folium.Map(location=[43.67146, -79.37515], zoom_start=11)
    folium.CircleMarker(
        [starting_lat, starting_long],
        radius=5,
        color='red',
         fill=True,
           fill_color='#3186cc',
           fill_opacity=0.7).add_to(map_clusters)



    # loop through dataframe
    for lat, lng, name, address in zip(df['latitude'], df['longitude'],  df['name'],df['address']):
       label = '{} {}'.format(name, address)
       label = folium.Popup(label, parse_html=True)
       folium.CircleMarker(
           [lat, lng],
           radius=5,
           popup=label,
           color='blue',
           fill=True,
           fill_color='#3186cc',
           fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

In [18]:
#initialize map with a default lat and long
def plot_stops_cluster(starting_lat, starting_long, df):
    map_clusters = folium.Map(location=[43.67146, -79.37515], zoom_start=11)
    folium.CircleMarker(
        [starting_lat, starting_long],
        radius=5,
        color='red',
         fill=True,
           fill_color='#3186cc',
           fill_opacity=0.7).add_to(map_clusters)



    # loop through dataframe
    df_excluded = df[df['labels'] !=0]
    df_included = df[df['labels'] ==0]
    for lat, lng, name, address in zip(df_included['latitude'], df_included['longitude'],  df_included['name'],df_included['address']):
       label = '{} {}'.format(name, address)
       label = folium.Popup(label, parse_html=True)
       folium.CircleMarker(
           [lat, lng],
           radius=5,
           popup=label,
           color='blue',
           fill=True,
           fill_color='#3186cc',
           fill_opacity=0.7).add_to(map_clusters)
    for lat, lng, name, address in zip(df_excluded['latitude'], df_excluded['longitude'],  df_excluded['name'],df_excluded['address']):
       label = '{} {}'.format(name, address)
       label = folium.Popup(label, parse_html=True)
       folium.CircleMarker(
           [lat, lng],
           radius=5,
           popup=label,
           color='#0C8E26',
           fill=True,
           fill_color='#0C8E26',
           fill_opacity=0.7).add_to(map_clusters)


    return map_clusters

# GENETIC ALGORITHM TO PLOT POINTS
* borrowed and slightly adapted from https://github.com/ZWMiller/PythonProjects/blob/master/genetic_algorithms/evolutionary_algorithm_traveling_salesman.ipynb

In [19]:
# TODO: cluster points

In [20]:
def get_dict_of_stop_coords(df,starting_lat, starting_long, key_to_use='poi_id'):
    '''
    create dictionary of stops with lat/long coords
    first stop is the starting point
    '''
   # print(f'starting point ({starting_lat}, {starting_long})')
    walk_stops = {}
    walk_stops[0] = (starting_lat, starting_long)
    for ix,row in df.iterrows():
        if key_to_use=='poi_id':
            walk_stops[row['poi_id']] = (row['latitude'], row['longitude'])
        else:
            #print(ix)
            walk_stops[ix+1] = (row['latitude'], row['longitude'])
        
    return walk_stops

In [21]:

def create_guess(walk_stops):
    """
    Creates a possible path between all cities, returning to the original.
    Point 0 is our starting point and must stay as our starting point
    Input: List of City IDs
    """
    guess = copy(walk_stops)
   # print(f"before shuffle {guess}")
    start = guess.pop(0)# save our starting point and remove from dict
    np.random.shuffle(guess)
   # print(f"after shuffle {guess}")
   # guess[0] = start
    guess.insert(0,start)
    #guess.append(guess[0])
    return list(guess)

#print(create_guess(list(walk_stops.keys())))


In [22]:

def create_generation(points, population=100):
    """
    Makes a list of guessed point orders given a list of point IDs.
    Input:
    points: list of point ids
    population: how many guesses to make
    """
    generation = [create_guess(points) for _ in range(population)]
    return generation

# test_generation = create_generation(list(walk_stops.keys()), population=10)
# for gen in test_generation:
#     print(gen)

In [23]:
def travel_time_between_points(point_1, point_2):
    '''
    pass in coords for 2 points
    calculate distance between them in meters and then estimate walking time
    '''
    # typical walkign speed is 1.4m/sec
    speed = 1.4
    #find dist between 2 points
    dist = geopy.distance.geodesic(point_1, point_2).meters
    # return guess speed in seconds
    return dist/speed

In [24]:
def fitness_score(guess, walk_stops):
    """
    Loops through the points in the guesses order and calculates
    how much distance the path would take to complete a loop.
    Lower is better.
    """
    score = 0
    for ix, point_id in enumerate(guess[:-1]):
        score += travel_time_between_points(walk_stops[point_id], walk_stops[guess[ix+1]])
    return score

def check_fitness(guesses, walk_stops):
    """
    Goes through every guess and calculates the fitness score. 
    Returns a list of tuples: (guess, fitness_score)
    """
    fitness_indicator = []
    for guess in guesses:
        fitness_indicator.append((guess, fitness_score(guess, walk_stops)))
    return fitness_indicator

#print(check_fitness(test_generation))

In [25]:
def get_breeders_from_generation(guesses, walk_stops, take_best_N=10, take_random_N=5, verbose=False, mutation_rate=0.1):
    """
    This sets up the breeding group for the next generation. You have
    to be very careful how many breeders you take, otherwise your
    population can explode. These two, plus the "number of children per couple"
    in the make_children function must be tuned to avoid exponential growth or decline!
    """
    # First, get the top guesses from last time
    fit_scores = check_fitness(guesses, walk_stops)
    sorted_guesses = sorted(fit_scores, key=lambda x: x[1]) # sorts so lowest is first, which we want
    new_generation = [x[0] for x in sorted_guesses[:take_best_N]] # takes top 5
    best_guess = new_generation[0] # best guess is the best score
    
    if verbose:
        # If we want to see what the best current guess is!
        print(best_guess)
    
    # Second, get some random ones for genetic diversity
    for _ in range(take_random_N):
        ix = np.random.randint(len(guesses))
        new_generation.append(guesses[ix])
        
    # No mutations here since the order really matters.
    # If we wanted to, we could add a "swapping" mutation,
    # but in practice it doesn't seem to be necessary
    
    np.random.shuffle(new_generation)
    return new_generation, best_guess

def make_child(parent1, parent2):
    """ 
    Take some values from parent 1 and hold them in place, then merge in values
    from parent2, filling in from left to right with cities that aren't already in 
    the child. 
    """
    list_of_ids_for_parent1 = list(np.random.choice(parent1, replace=False, size=len(parent1)//2))
   # print(list_of_ids_for_parent1)
    child = [-99 for _ in parent1] # fill with placeholders so now all -99 values can be replaced with genes from parent 2
    
    for ix in range(0, len(list_of_ids_for_parent1)):
        child[ix] = parent1[ix]

    for ix, gene in enumerate(child):
 
        if gene == -99:
            for gene2 in parent2:
                if gene2 not in child:
                    child[ix] = gene2
                    break
    #child[-1] = child[0]
    return child

def make_children(old_generation, children_per_couple=1):
    """
    Pairs parents together, and makes children for each pair. 
    If there are an odd number of parent possibilities, one 
    will be left out. 
    
    Pairing happens by pairing the first and last entries. 
    Then the second and second from last, and so on.
    """
  #  print(old_generation)
    mid_point = len(old_generation)//2
#    print(mid_point)
    next_generation = [] 
    
    for ix, parent in enumerate(old_generation[:mid_point]):
        for _ in range(children_per_couple):
            next_generation.append(make_child(parent, old_generation[-ix-1]))
   # print(next_generation)
    return next_generation

In [26]:
def evolve_to_solve(current_generation, max_generations, take_best_N, take_random_N,
                    mutation_rate, children_per_couple, print_every_n_generations, walk_stops, verbose=False):
    """
    Takes in a generation of guesses then evolves them over time using our breeding rules.
    Continue this for "max_generations" times.
    Inputs:
    current_generation: The first generation of guesses
    max_generations: how many generations to complete
    take_best_N: how many of the top performers get selected to breed
    take_random_N: how many random guesses get brought in to keep genetic diversity
    mutation_rate: How often to mutate (currently unused)
    children_per_couple: how many children per breeding pair
    print_every_n_geneartions: how often to print in verbose mode
    verbose: Show printouts of progress
    Returns:
    fitness_tracking: a list of the fitness score at each generations
    best_guess: the best_guess at the end of evolution
    """
    fitness_tracking = []
    for i in range(max_generations):
        if verbose and not i % print_every_n_generations and i > 0:
            print("Generation %i: "%i, end='')
            print(len(current_generation))
            print("Current Best Score: ", fitness_tracking[-1])
            is_verbose = True
        else:
            is_verbose = False
        breeders, best_guess = get_breeders_from_generation(current_generation, walk_stops, 
                                                            take_best_N=take_best_N, take_random_N=take_random_N, 
                                                            verbose=is_verbose, mutation_rate=mutation_rate)
        fitness_tracking.append(fitness_score(best_guess, walk_stops))
        current_generation = make_children(breeders, children_per_couple=children_per_couple)
    
    return fitness_tracking, best_guess

#current_generation = create_generation(list(walk_stops.keys()),population=500)
# for gen in current_generation:
#     print(gen)

In [27]:
def add_stop_order_to_df(df, order_guess, key_to_use='poi_id'):
    df['order'] =0
    cnt = 1
    for ix in order_guess[1:]:
        if key_to_use=='poi_id':
            index = df[df_filtered['poi_id'] ==ix].index
        else:
            index = ix-1
        df.iloc[index,-1]=cnt
        cnt +=1
    return df

In [28]:
def plot_guess(city_coordinates, guess, guess_in_title=True):
    """
    Takes the coordinates of the cities and the guessed path and
    makes a plot connecting the cities in the guessed order
    Input:
    city_coordinate: dictionary of city id, (x,y)
    guess: list of ids in order
    """
    plot_cities(city_coordinates, guess=guess)
    for ix, current_city in enumerate(guess[:-1]):
        x = [city_coordinates[guess[ix]][0],city_coordinates[guess[ix+1]][0]]
        y = [city_coordinates[guess[ix]][1],city_coordinates[guess[ix+1]][1]]
        plt.plot(x,y,'c--',lw=1)
    plt.scatter(city_coordinates[guess[0]][0],city_coordinates[guess[0]][1], marker='x', c='b')   
    if guess_in_title:
        plt.title("Current Guess: [%s]"%(','.join([str(x) for x in guess])))
    else:
        print("Current Guess: [%s]"%(','.join([str(x) for x in guess])))
    


In [29]:
def plot_cities(city_coordinates, annotate=True, guess=None):
    """
    Makes a plot of all cities.
    Input: city_coordinates; dictionary of all cities and their coordinates in (x,y) format
    """
    names = []
    x = []
    y = []
    plt.figure(dpi=250, figsize=(20,20))
    for key in list(city_coordinates.keys()):
        names.append(key)
        x.append(city_coordinates[key][0])
        y.append(city_coordinates[key][1])
        if annotate:
            if guess:
                plt.annotate(guess.index(key), xy=(city_coordinates[key][0], city_coordinates[key][1]), xytext=(20, -20),
                            textcoords='offset points', ha='right', va='bottom',
                            bbox=dict(boxstyle='round,pad=0.5', fc='w', alpha=0.5),
                            arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
            else:
                plt.annotate(ix, xy=(city_coordinates[key][0], city_coordinates[key][1]), xytext=(20, -20),
                            textcoords='offset points', ha='right', va='bottom',
                            bbox=dict(boxstyle='round,pad=0.5', fc='w', alpha=0.5),
                            arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
    plt.scatter(x,y,c='r',marker='o')

In [30]:
def make_fitness_tracking_plot(fitness_tracking):
    """
    Given a list of fitness scores, plot it versus the generation number
    """
    plt.figure(dpi=150)
    plt.plot(range(len(fitness_tracking)), fitness_tracking)
    plt.ylabel("Fitness Score")
    plt.xlabel("Generation")
    plt.title("Fitness Evolution");

#make_fitness_tracking_plot(fitness_tracking)

In [31]:
def plot_stops_with_order(df, starting_lat, starting_long):
    map_clusters = folium.Map(location=[43.67146, -79.37515], zoom_start=11)
    folium.CircleMarker(
        [starting_lat, starting_long],
        radius=5,
        color='red',
         fill=True,
           fill_color='#3186cc',
           fill_opacity=0.7).add_to(map_clusters)



    # loop through dataframe
    for lat, lng, name, address,order in zip(df['latitude'], df['longitude'],  df['name'],df['address'], df['order']):
        label = '{} {}'.format(name, address)
        label = folium.Popup(label, parse_html=True)
        folium.map.Marker(
        [lat, lng],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
           # html='<b><div style="color:red,font-size: 10pt">{}</div></b>'.format(order),
            html='<b><div style="color:red">{}</div></b>'.format(order),
            )
        ).add_to(map_clusters)

    return map_clusters

In [32]:
def find_optimal_route(df, starting_lat, starting_long, method='genetic'):
    '''
    Possible methods: genetic or google optimisation tools
    '''
    
    if method == 'genetic':
        walk_stops = get_dict_of_stop_coords(df, starting_lat, starting_long)
        current_generation = create_generation(list(walk_stops.keys()),population=500)
        fitness_tracking, best_guess = evolve_to_solve(current_generation, 100, 150, 70, 0.5, 3, 5, walk_stops, verbose=True)
        plot_guess(walk_stops, best_guess)
        # add order to df.
        df=add_stop_order_to_df(df, best_guess)
        make_fitness_tracking_plot(fitness_tracking)
        map_stops = plot_stops_with_order(df, starting_lat, starting_long)
        total_dist=0
    else:
        #method='google'
        walk_stops = get_dict_of_stop_coords(df, starting_lat, starting_long, key_to_use='count')
#         print(df.shape)
#         print(len(walk_stops))
        dist_matrix=make_dist_matrix(len(walk_stops))
        order_guess, total_dist=create_routing_model(walk_stops,dist_matrix)

        df=add_stop_order_to_df(df, order_guess,key_to_use='count')
        map_stops = plot_stops_with_order(df, starting_lat, starting_long)
    return map_stops, order_guess, df, walk_stops, total_dist

# Try with Google OR Tools

In [33]:
def get_google_key():
    load_dotenv(find_dotenv())
    # load environment variables
    SECRET_KEY = os.getenv("GOOGLE_KEY")
    gmaps = googlemaps.Client(key=SECRET_KEY)
    return gmaps

In [34]:
# Distance callback
def create_distance_callback(dist_matrix):
    # Create a callback to calculate distances between cities.

    def distance_callback(from_node, to_node):
        # return int(dist_matrix[from_node][to_node])
        #print(dist_matrix[from_node][to_node])
        return dist_matrix[from_node][to_node]


    return distance_callback

In [35]:
def make_dist_matrix(num_rows):
    #my_dist_matrix = np.zeros((len(walk_stops), len(walk_stops)))
    dist_matrix = np.zeros((num_rows,num_rows))
    return dist_matrix
#dist_matrix = gmaps.distance_matrix(walk_stops.values(),walk_stops.values())
#my_dist_matrix[0,0]

In [36]:
def euclid_distance(x1, y1, x2, y2):
    # Euclidean distance between points.
    dist = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    return dist

In [37]:
def create_distance_matrix(locations):
    # Create the distance matrix.
    gmaps = get_google_key()
    size = len(locations)
    dist_matrix = {}

    for from_node in locations.keys():
        dist_matrix[from_node] = {}
        for to_node in locations.keys():
            x1 = locations.get(from_node)[0]
            y1 = locations.get(from_node)[1]
            x2 = locations.get(to_node)[0]
            y2 = locations.get(to_node)[1]
           # print(x1,y1, x2, y2)
           # dist_matrix[from_node][to_node] = euclid_distance(x1, y1, x2, y2)
            #dist_matrix[from_node][to_node]  = gmaps.distance_matrix((x1,y1), (x2,y2), mode='walking')["rows"][0]["elements"][0]["distance"]["value"]
            dist_matrix[from_node][to_node]  = geopy.distance.geodesic((x1,y1), (x2,y2)).meters
    return dist_matrix

In [38]:
#dist_matrix
#https://developers.google.com/optimization/routing/tsp
def create_routing_model(walk_stops, dist_matrix ):
    tsp_size = len(walk_stops)
    num_routes = 1
    depot = 0
    best_guess=[]

    # Create routing model
    if tsp_size > 0:
        # Routing Model is the solver
        # depot would be start and end of route -- n/a for us
        routing = pywrapcp.RoutingModel(tsp_size, num_routes, depot)
        search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
        # Create the distance callback.
        dist_matrix=create_distance_matrix(walk_stops)
        dist_callback = create_distance_callback(dist_matrix)
        #print(dist_matrix)
        routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
        # Solve the problem.
        assignment = routing.SolveWithParameters(search_parameters)
        if assignment:
            # Solution distance.
            print ("Total distance: " + str(assignment.ObjectiveValue()) + " meters\n")
            total_dist = assignment.ObjectiveValue()
           # logging.debug("Total distance: " + str(assignment.ObjectiveValue()) + " meters\n")
            # Display the solution.
            # Only one route here; otherwise iterate from 0 to routing.vehicles() - 1
            route_number = 0
            index = routing.Start(route_number) # Index of the variable for the starting node.
            route = ''
            while not routing.IsEnd(index):
                # Convert variable indices to node indices in the displayed route.
                print(routing.IndexToNode(index))
                best_guess.append(routing.IndexToNode(index))
                route += str(walk_stops[routing.IndexToNode(index)]) + ' -> '
                index = assignment.Value(routing.NextVar(index))
            route += str(walk_stops[routing.IndexToNode(index)])
            print ("Route:\n\n" + route)
            #logging.debug("Route:\n\n" + route)
        else:
            print( 'No solution found.')
            #logging.debug("No solution found")
    else:
        print ('Specify an instance greater than 0.')
    return best_guess, total_dist

# RouteTesting

In [48]:
def run_test_suite():
    starting_points = [(43.647273,-79.386560),(43.668697,-79.394173),(43.668752,-79.305300),(43.683117, -79.418790)]
    durations = [1]
    pts_per_hour=[12]
    max_dist_per_hour=[500, 1000]
    user_profiles = [1,2,3]
    # TODO: update to new user profiels
    #user_interests = ['build_century_1700.0', 'poi_type_simple_Plaque']
    similarity_methods=['cosine_similarity','spatial']
    
    walknum=0
    # store in log

    #similarity_method ='spatial' # 'co-sine'
    optimizer = 'or'
    for similarity_method in similarity_methods:
        for pt in starting_points:
            for dur in durations:
                for maxdist in max_dist_per_hour:
                    for pph in pts_per_hour:
                        for u in user_profiles:
                            num_points = dur * pph
                            max_distance=dur * maxdist

                            df_user = get_user_profile_old(u)
                            df_poi = get_pois_as_df()
                            df_features=df_poi.copy()
                            poi_mapper.fit(df_features)
                            df_features= poi_mapper.transform(df_features)
                            df_poi=find_similarity(df_features, df_user, df_poi,sim_method=similarity_method)
                            df_filtered = find_points_in_area(df_poi, pt[0], pt[1], num_points, max_distance)
                            
                            # add clustering
                            df_filtered = cluster_stops(df_filtered, num_points)
                            df_filtered=df_filtered.reset_index()
                            # find optimal route
                            map_stops, guess, df_filtered, walk_stops, total_dist=find_optimal_route(df_filtered,pt[0], pt[1], method='google')

                            stops_ordered = []
                            for stop in guess:
                                stops_ordered.append(walk_stops[stop])
                            df_filtered.sort_values("order", inplace=True, ascending=True)

                            # get info about the generated walk
                            num_art=len(df_filtered[df_filtered['poi_type_simple']=='Art'])
                            num_building = len(df_filtered[df_filtered['poi_type_simple']=='Building'])
                            num_plaque = len(df_filtered[df_filtered['poi_type_simple']=='Plaque'])

                            stops = str(df_filtered['poi_id'].values)

                            guess2=map(str,guess) 
                            guess2='-'.join(guess2)
                            walknum += 1
                            logging.debug(f"{walknum}, {similarity_method}, {optimizer}, {maxdist},{pt[0]}, {pt[1]}, {dur},\
                                          {num_points}, {total_dist}, {total_dist/num_points}, {0}, {guess2}, {num_art}, {num_building}, \
                                          {num_plaque}, {stops},{df_filtered['sim_rating'].mean()}, \
                                            {df_filtered['sim_rating'].max()}, {u}, simplified prefs ")
                            df_filtered.to_csv(f'../logs/walk_{walknum}.csv')
                            #df_user.to_csv(f'../logs/userprefs_{walknum}.csv')

In [45]:
#run_test_suite()

cosine_similarity
0
12 5
Total distance: 1103 meters

0
11
12
10
9
8
7
6
4
5
3
1
2
Route:

(43.647273, -79.38656) -> (43.64806, -79.38998) -> (43.64802, -79.39005) -> (43.64809, -79.39012) -> (43.64813, -79.39011) -> (43.64823, -79.39012) -> (43.64827, -79.39013) -> (43.64831, -79.3901) -> (43.64844, -79.39015) -> (43.64841, -79.39019) -> (43.64854, -79.3911) -> (43.64765, -79.39039) -> (43.64584, -79.39065) -> (43.647273, -79.38656)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


cosine_similarity
0
12 5
Total distance: 1760 meters

0
7
1
2
5
4
9
8
6
11
12
3
10
Route:

(43.647273, -79.38656) -> (43.64743, -79.38641) -> (43.647353, -79.385716) -> (43.64519, -79.383602) -> (43.644514, -79.382232) -> (43.644989, -79.381392) -> (43.645602, -79.381489) -> (43.645706, -79.381157) -> (43.6467, -79.38221) -> (43.647562, -79.381049) -> (43.64811, -79.38133) -> (43.648711, -79.386515) -> (43.646912, -79.387642) -> (43.647273, -79.38656)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


cosine_similarity
0
12 5
Total distance: 1699 meters

0
10
6
9
5
12
3
11
7
8
4
1
2
Route:

(43.647273, -79.38656) -> (43.64757, -79.38684) -> (43.6486892700195, -79.3854370117188) -> (43.6509568, -79.3853608) -> (43.651317, -79.3869) -> (43.6509, -79.38711) -> (43.650422051, -79.3870218498) -> (43.6494, -79.38866) -> (43.64956, -79.38964) -> (43.64745, -79.3876) -> (43.647155, -79.387475) -> (43.646162378, -79.3889347303) -> (43.6467072718, -79.3864537867) -> (43.647273, -79.38656)


...

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


0
12 5
Total distance: 4131 meters

0
3
5
8
10
12
1
6
2
11
7
9
4
Route:

(43.683117, -79.41879) -> (43.6833241951, -79.4187878551) -> (43.6825564393, -79.4201273017) -> (43.6822924568, -79.4240444491) -> (43.692, -79.42006) -> (43.69158, -79.41806) -> (43.68941, -79.41666) -> (43.6857216141, -79.4075167985) -> (43.685073908, -79.410731333) -> (43.68399, -79.41192) -> (43.6839008332, -79.4127883698) -> (43.6835910842, -79.4151302611) -> (43.6812208373, -79.4180535667) -> (43.683117, -79.41879)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


0
12 5
Total distance: 4612 meters

0
6
9
2
12
10
5
3
4
1
8
11
7
Route:

(43.683117, -79.41879) -> (43.679872, -79.417645) -> (43.6768765, -79.4140558) -> (43.67789, -79.41327) -> (43.68399, -79.41192) -> (43.684581, -79.411513) -> (43.68695, -79.41756) -> (43.68941, -79.41666) -> (43.69158, -79.41806) -> (43.688977, -79.420466) -> (43.68173, -79.425399) -> (43.6822069, -79.4244543) -> (43.682132, -79.423783) -> (43.683117, -79.41879)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


### Test 1 walk

In [ ]:
# king/simco
# starting_lat =43.647273
# starting_long=-79.386560
# Avenue/bloor
starting_lat =43.668697
starting_long=-79.394173
# #lakeshort / woodbine
# starting_lat =43.668752
# starting_long=-79.305300
# #st clair / bathhurst
# starting_lat =43.683117
# starting_long=-79.418790

# walk_duration = 1 # in hours
# num_points = 12
# max_distance = 1000 # meters
# num_pois_visit = 20
duration = 1.5
pts_per_hour = 12
max_dist_per_hour = 1000 # meters
num_points = duration * pts_per_hour
max_distance=duration * max_dist_per_hour
user_profile=3
#user_interests = ['build_century_1700.0', 'poi_type_simple_Plaque']

In [ ]:

df_poi = get_pois_as_df()
df_features=df_poi.copy()
poi_mapper.fit(df_features)
df_features= poi_mapper.transform(df_features)
avail_interests = list(df_features.columns)
df_user = get_user_profile(3)#avail_interests, user_interests)

df_poi=find_similarity(df_features, df_user, df_poi) #, sim_method='hamming') #hamming')#, sim_method='spatial')
# print(df_poi.shape)

df_filtered = find_points_in_area(df_poi, starting_lat, starting_long, num_points, max_distance)#,inflate_amount=1)
#df_filtered
#plot_stops(starting_lat, starting_long, df_filtered)


In [ ]:
df_filtered,my_plot = cluster_stops(df_filtered, num_points)
df_filtered=df_filtered.reset_index()

In [ ]:

map_stops, guess, df_filtered, walk_stops, total_dist=find_optimal_route(df_filtered, starting_lat, starting_long, method='google')


In [ ]:
df_filtered[['name', 'address','latitude', 'longitude', 'poi_type_simple', 'sim_rating']]

#str(df_filtered['poi_type_simple'].value_counts())
num_art=len(df_filtered[df_filtered['poi_type_simple']=='Art'])
num_building = len(df_filtered[df_filtered['poi_type_simple']=='Building'])
num_plaque = len(df_filtered[df_filtered['poi_type_simple']=='Plaque'])

stops = str(df_filtered['poi_id'].values)
num_art, num_building, num_plaque,stops

In [ ]:
#(0, 6, 1, '[4576 4668 4655 4582 4720 5142 5012]') Hamming

In [ ]:
# map_stops = plot_stops(starting_lat, starting_long, df_filtered)
# map_stops

In [ ]:
# map_stops=find_optimal_route(df_filtered, starting_lat, starting_long,method='genetic')
# map_stops

In [ ]:
# map_stops=find_optimal_route(df_filtered, starting_lat, starting_long, method='google')
# map_stops

map_stops, guess, df_filtered, walk_stops, total_dist=find_optimal_route(df_filtered, starting_lat, starting_long, method='google')
